In [1]:
import DCCRN
import torch
from asteroid.data import DNSDataset
import os
import torch.nn.functional as F
import time
import torch.nn as nn
import shutil
import numpy as np
from framework import ABF, MultiResolutionSTFTLoss, SPKDLoss, build_review_kd, ReviewKD, hcl
from asteroid.models import DCCRNet
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import config as cfg
from tools_for_model import near_avg_index, max_index, min_index, Bar, cal_pesq, cal_stoi
from dataloader import create_dataloader
import inspect
import os
import feature_extraction
from asteroid.models import DCCRNet_mini
  
os.environ["CUDA_VISIBLE_DEVICES"]="2"


In [2]:
#model = DCCRN.DCCRN(rnn_units=cfg.rnn_units_student, masking_mode=cfg.masking_mode, use_clstm=cfg.use_clstm,kernel_num=cfg.kernel_num_student)
#model = DCCRN.DCCRN(rnn_units=cfg.rnn_units, masking_mode=cfg.masking_mode, use_clstm=cfg.use_clstm,kernel_num=cfg.kernel_num)
#data_set =  DNSDataset("/root/NTH_student/Speech_Enhancement_new/asteroid/egs/dns_challenge/baseline/data")
#model = DCCRNet.from_pretrained('JorisCos/DCCRNet_Libri1Mix_enhsingle_16k')
model = DCCRNet_mini(architecture='DCCRN-CL-test')
train_dataset =  DNSDataset("/root/NTH_student/test_loader")
train_loader = create_dataloader(mode='train',dataset=train_dataset)

In [3]:
# tạo sample để input mode. cái này random đại thui
# input_size = model.stft.weight.size(1)
# tạo cái batch có n= 5 đi
# input = torch.randn(64, input_size)

In [4]:
# # Tạo list chứa các feature map do encoder output ra 
# # dùng list vì lát nữa feature map của lstm với decoder bỏ vô đây luôn. ở đây demo cái encoder thui nha. 
# feature_maps_encoder = []
# feature_maps_decoder = []
# # Tạo hàm hook, hàm này sẽ thực thi khi encoder thực hiện forward.
# # output của encoder sẽ được append vào trong encoder_feature_map(một biến toàn cục)
# def encoder_hook(module, input, output):
#     feature_maps_encoder.append(output)
# # đăng kí forward hook. kiểu "ê khi nào encoder thực hiện foward xong thì chạy hàm encoder_hook nha"

# [model.encoder[i].register_forward_hook(encoder_hook) for i in range(len(model.encoder))]

In [5]:
# def decoder_hook(module, input, output):
#     feature_maps_decoder.append(output)

# for i in range(0,len(model.decoder)):
#     model.decoder[i].register_forward_hook(decoder_hook)

In [6]:
# def enhance_hook(module, input, output):
#     feature_maps.append(output)

# model.enhance.register_forward_hook(enhance_hook)

In [7]:
X,Y = 0,0
i = 0
for x,y,_ in Bar(train_loader):
    if i == 1: break
    X=x
    Y=y
    i += 1

# try :
#     model(X)
#     print('au de')
# except:
#     print('failed roihuhu')


 64/600: [==>...............................................] - ETA 75.5s

In [8]:
feature_maps_extraction = feature_extraction.DCCRNet(model)
feature_maps = feature_maps_extraction.extract_feature_maps(X)

In [9]:
len(feature_maps["clstm_real"])

1

In [10]:
len(feature_maps["clstm_img"])

1

In [11]:
feature_maps["clstm_real"][0].shape

torch.Size([32, 471, 32])

In [12]:
len(feature_maps)

4

In [13]:
len(feature_maps['encoder'])

6

In [14]:
feature_maps_encod = feature_maps['encoder']
for i in range(6):
    print(feature_maps_encod[i].shape)

torch.Size([32, 8, 128, 476])
torch.Size([32, 16, 64, 475])
torch.Size([32, 32, 32, 474])
torch.Size([32, 64, 16, 473])
torch.Size([32, 64, 8, 472])
torch.Size([32, 64, 4, 471])


In [15]:
feature_maps_encod = feature_maps['decoder']
for i in range(6):
    print(feature_maps_encod[i].shape)

torch.Size([32, 64, 4, 471])
torch.Size([32, 64, 8, 472])
torch.Size([32, 64, 16, 473])
torch.Size([32, 32, 32, 474])
torch.Size([32, 16, 64, 475])
torch.Size([32, 8, 128, 476])


In [16]:
feature_maps['encoder'][0].shape #[64, 64, 4, 483]


torch.Size([32, 8, 128, 476])

In [17]:
feature_maps['decoder'][2].shape #[64, 2, 256, 484]

torch.Size([32, 64, 16, 473])

## encoder

In [18]:
in_channels = [8, 16, 32, 64, 64, 64]
out_channels = [32, 64, 128, 256, 256, 256]

# shapes = [1,4,8,16,32]
# out_shapes = [1,4,8,16,32]
shapes = [4,8,16,32,64,128]
out_shapes = [4,8,16,32,64,128]
abfs = nn.ModuleList()
mid_channel = min(512, in_channels[-1])
for idx, in_channel in enumerate(in_channels):
    abfs.append(ABF(in_channel, mid_channel, out_channels[idx], idx < len(in_channels)-1))
abfs = abfs[::-1]

In [19]:
x = feature_maps['encoder'][::-1] #encoder


results = []
out_features, res_features = abfs[0](x[0].cuda(), out_shape=out_shapes[0],feature_type="encoder")
results.append(out_features)
for feature, abf, shape, out_shape in zip(x[1:], abfs[1:], shapes[1:], out_shapes[1:]):
        out_features, res_features = abf(feature.cuda(), res_features, shape, out_shape,"encoder")
        print("1")
        results.insert(0, out_features)

1
1
1
1
1


In [20]:
for i in range(len(results)):
    print(results[i].shape)

torch.Size([32, 32, 128, 476])
torch.Size([32, 64, 64, 475])
torch.Size([32, 128, 32, 474])
torch.Size([32, 256, 16, 473])
torch.Size([32, 256, 8, 472])
torch.Size([32, 256, 4, 471])


In [21]:
# import torch

# re = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
# im = torch.tensor([[5.0, 6.0], [7.0, 8.0]])

# complex_tensor = torch.view_as_complex(torch.stack([re, im], dim=-1))
# h = torch.stack([re, im])
# h.shape

## Decoder

In [22]:
in_channels = [8, 16, 32, 64, 64, 64]
out_channels = [32, 64, 128, 256, 256, 256]

# shapes = [1,4,8,16,32]
# out_shapes = [1,4,8,16,32]
shapes = [4,8,16,32,64,128]
out_shapes = [4,8,16,32,64,128]
abfs = nn.ModuleList()
mid_channel = min(512, in_channels[-1])
for idx, in_channel in enumerate(in_channels):
    abfs.append(ABF(in_channel, mid_channel, out_channels[idx], idx < len(in_channels)-1))
abfs = abfs[::-1]

In [23]:
x = feature_maps['decoder']
results = []
out_features, res_features = abfs[0](x[0].cuda(), out_shape=out_shapes[0],feature_type="decoder")
results.append(out_features)
for feature, abf, shape, out_shape in zip(x[1:], abfs[1:], shapes[1:], out_shapes[1:]):
        out_features, res_features = abf(feature.cuda(), res_features, shape, out_shape,feature_type="decoder")
        print("1")
        results.append(out_features)

1
1
1
1
1


In [24]:
for i in range(6):
    print(results[i].shape)

torch.Size([32, 256, 4, 471])
torch.Size([32, 256, 8, 472])
torch.Size([32, 256, 16, 473])
torch.Size([32, 128, 32, 474])
torch.Size([32, 64, 64, 475])
torch.Size([32, 32, 128, 476])


In [25]:
len(results)

6

In [36]:
loss = hcl(results[1],results[1],'encoder')

# CLSTM

In [38]:
stu_img = feature_maps['clstm_img'][0]
stu_img.shape

torch.Size([32, 471, 128])

In [39]:
l  = hcl(feature_maps['clstm_img'][0],feature_maps['clstm_img'][0],'lstm')

ValueError: Input dimension should be at least 3

In [28]:
stu_rea = feature_maps['clstm_real'][0]

In [29]:
model = DCCRNet.from_pretrained('JorisCos/DCCRNet_Libri1Mix_enhsingle_16k')
feature_maps_extraction = feature_extraction.DCCRNet(model)
feature_maps = feature_maps_extraction.extract_feature_maps(X)

In [30]:
tea_img = feature_maps['clstm_img'][0]

In [31]:
tea_rea = feature_maps['clstm_real'][0]

In [32]:
kd_loss = SPKDLoss(stu_rea, tea_rea,reduction='batchmean')
loss = kd_loss()
loss

tensor(0.0015, grad_fn=<DivBackward0>)

In [33]:
kd_loss = SPKDLoss(stu_img, tea_img,reduction='batchmean')
loss = kd_loss()
loss

tensor(0.0011, grad_fn=<DivBackward0>)

In [34]:
#feature_maps.clear()
feature_maps_extraction.remove_hook()